In [3]:
import numpy as np
import pandas as pd

In [14]:
def create_synthetic_dataset(N, shuffle):
    # random array
    prescription = np.full(N, fill_value='acetominophen', dtype='U20')
    prescription[:N//2] = 'ibuprofen'
    # half of the set is acetominophen and the other half is ibuprofen
    # we shuffle the list
    np.random.shuffle(prescription)
    print("prescription : ",prescription)
    

In [17]:
create_synthetic_dataset(10, False)

['acetominophen' 'acetominophen' 'ibuprofen' 'acetominophen' 'ibuprofen'
 'acetominophen' 'ibuprofen' 'ibuprofen' 'ibuprofen' 'acetominophen']
